In [ ]:
def filter_correlations(dataframe,
                            max_corr_threshold,
                            mean_corr_threshold,
                            features_to_keep):
        """
        For every feature
        1) skips all the next steps if in features_to_keep
        2) counts mean and max correlations for Q3 quantile
        3) compares max and mean values with max_corr_threshold and mean_corr_threshold, respectively
        4) deletes if at least one number is bigger than the threshold
        :param dataframe: table in the format of DataFrame object.
        :param max_corr_threshold: the minimal level of correlation considered as "strong".
        :param mean_corr_threshold: the minimal level of mean correlation considered as "strong".
        :param features_to_keep: set of important chemical features to keep in any case (includes a target feature).
        :return: a DataFrame object M columns (where M <= N) with chemical descriptors from Rdkit
        + 1 column (target feature) with disease categories.
        """

        columns_to_drop = features_to_keep + ['disease category', 'phase', 'status', 'iso_smiles']
        descriptors = dataframe.drop(columns_to_drop, axis=1)


        corr_matrix = descriptors.corr().abs()
        Columns = corr_matrix.columns

        correlated_descriptors = []

        for i in range(len(corr_matrix.columns)):
          for j in range(i):
            if corr_matrix.iloc[i, j] > max_corr_threshold:
              column_mean_corr_1 = np.mean(corr_matrix[Columns[i]])
              column_mean_corr_2 = np.mean(corr_matrix[Columns[j]])
              column_to_drop = i*(column_mean_corr_1 > mean_corr_threshold and column_mean_corr_1 > column_mean_corr_2) + j*(column_mean_corr_2 > mean_corr_threshold and column_mean_corr_2 > column_mean_corr_1)
              correlated_descriptors.append(Columns[column_to_drop])

        corr_filtered_dataset_with_descriptors = dataframe.drop(correlated_descriptors, axis=1)

        corr_filtered_dataset_with_descriptors['iso_smiles'] = dataframe['iso_smiles']
        corr_filtered_dataset_with_descriptors['phase'] = dataframe['phase']
        corr_filtered_dataset_with_descriptors['status'] = dataframe['status']
        corr_filtered_dataset_with_descriptors['disease category'] = dataframe['disease category']

        return corr_filtered_dataset_with_descriptors